In [1]:
cd

/Users/emiljaffal


In [2]:
import pandas as pd

file_path = "desktop/data.xlsx"
data = pd.read_excel(file_path)
data

,Formula,Entry,Entry prototype,Structure class,Inv. center,"Unit cell volume, A^3",Density (calc.) [Mg m‰…]
0,CeGa,261037,1,1,1,210.590,6.62
1,CeGa,526690,1,1,1,212.790,6.55
2,CeGa,450659,1,1,1,212.790,6.55
3,CeGa,450525,1,1,1,215.020,6.48
4,CeGa,528215,1,1,1,215.030,6.48
...,...,...,...,...,...,...,...
1447,NiPt,1627941,10,4,1,26.500,15.91
1448,NiPt,455847,10,4,1,26.473,15.92
1449,SnYb,450594,10,4,1,54.147,8.95
1450,SnYb,1823352,10,4,1,54.800,8.84


In [3]:
cd desktop/elemnet/elemnet

/Users/emiljaffal/Desktop/ElemNet/elemnet


In [4]:
#had to update composition.py, data.py, conversions.py from matminer... code not maintained in accordance to updated submodules:
    #from pymatgen.core.composition import Composition #UPDATE
    #from pymatgen.ext.matproj import MPRester

import re, numpy as np, os, sys, pandas
from pymatgen.core.composition import Composition 
import data_utils
import magpie
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.utils.conversions import str_to_composition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

/Users/emiljaffal/anaconda3/envs/elemnet/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [5]:
formula_unitcell = data[['Formula', 'Unit cell volume, A^3']]
formula_unitcell.rename(columns={'Formula':'composition'}, inplace=True)
formula_unitcell

,composition,"Unit cell volume, A^3"
0,CeGa,210.590
1,CeGa,212.790
2,CeGa,212.790
3,CeGa,215.020
4,CeGa,215.030
...,...,...
1447,NiPt,26.500
1448,NiPt,26.473
1449,SnYb,54.147
1450,SnYb,54.800


In [6]:
formula_unitcell['comp_obj'] = formula_unitcell['composition'].apply(lambda x: Composition(x))
formula_unitcell

,composition,"Unit cell volume, A^3",comp_obj
0,CeGa,210.590,"(Ce, Ga)"
1,CeGa,212.790,"(Ce, Ga)"
2,CeGa,212.790,"(Ce, Ga)"
3,CeGa,215.020,"(Ce, Ga)"
4,CeGa,215.030,"(Ce, Ga)"
...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)"
1448,NiPt,26.473,"(Ni, Pt)"
1449,SnYb,54.147,"(Sn, Yb)"
1450,SnYb,54.800,"(Sn, Yb)"


In [7]:
formula_unitcell['pretty_comp'] = formula_unitcell['comp_obj'].apply(lambda x: x.reduced_formula)
formula_unitcell

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp
0,CeGa,210.590,"(Ce, Ga)",CeGa
1,CeGa,212.790,"(Ce, Ga)",CeGa
2,CeGa,212.790,"(Ce, Ga)",CeGa
3,CeGa,215.020,"(Ce, Ga)",CeGa
4,CeGa,215.030,"(Ce, Ga)",CeGa
...,...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)",NiPt
1448,NiPt,26.473,"(Ni, Pt)",NiPt
1449,SnYb,54.147,"(Sn, Yb)",YbSn
1450,SnYb,54.800,"(Sn, Yb)",YbSn


In [8]:
#formula_unitcell.drop_duplicates('pretty_comp', keep='first', inplace=True)
#print('Reduced dataset to %d entries'%len(formula_unitcell))

In [9]:
formula_unitcell['nelems'] = formula_unitcell['comp_obj'].apply(lambda x: len(x))
formula_unitcell

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp,nelems
0,CeGa,210.590,"(Ce, Ga)",CeGa,2
1,CeGa,212.790,"(Ce, Ga)",CeGa,2
2,CeGa,212.790,"(Ce, Ga)",CeGa,2
3,CeGa,215.020,"(Ce, Ga)",CeGa,2
4,CeGa,215.030,"(Ce, Ga)",CeGa,2
...,...,...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)",NiPt,2
1448,NiPt,26.473,"(Ni, Pt)",NiPt,2
1449,SnYb,54.147,"(Sn, Yb)",YbSn,2
1450,SnYb,54.800,"(Sn, Yb)",YbSn,2


In [10]:
formula_unitcell.groupby('nelems').count()

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp
nelems,,,,
2,1452,1452,1452,1452


In [11]:
formula_unitcell['comp_dict'] = formula_unitcell['pretty_comp'].apply(lambda x: data_utils.parse_formula(x))
formula_unitcell

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp,nelems,comp_dict
0,CeGa,210.590,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}"
1,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}"
2,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}"
3,CeGa,215.020,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}"
4,CeGa,215.030,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}"
...,...,...,...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}"
1448,NiPt,26.473,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}"
1449,SnYb,54.147,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}"
1450,SnYb,54.800,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}"


In [12]:
elements_present = set()
for i, row in formula_unitcell.iterrows():
    elements_present |= set(row['comp_dict'].keys())
print (elements_present)

{'Rh', 'Ho', 'Ni', 'Tb', 'In', 'Fe', 'Si', 'Sn', 'Dy', 'Er', 'Sm', 'Y', 'Pt', 'Lu', 'Sb', 'Gd', 'U', 'Eu', 'Ir', 'Yb', 'Tm', 'Co', 'La', 'Ge', 'Pr', 'Os', 'Pd', 'Th', 'Nd', 'Sc', 'Ga', 'Ru', 'Ce'}


In [13]:
elements = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
            'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
            'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 
            'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
            'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
            'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
            'Ds', 'Rg', 'Cn']


elements_tl = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

elem_pos = dict()
i=0
for el in elements:
   elem_pos[el] = i
   i+=1

In [14]:
print (elements_present, len(elements_present))
print ([e for e in elements_present if e not in elements_tl])
print ([e for e in elements_tl if e not in elements_present])

{'Rh', 'Ho', 'Ni', 'Tb', 'In', 'Fe', 'Si', 'Sn', 'Dy', 'Er', 'Sm', 'Y', 'Pt', 'Lu', 'Sb', 'Gd', 'U', 'Eu', 'Ir', 'Yb', 'Tm', 'Co', 'La', 'Ge', 'Pr', 'Os', 'Pd', 'Th', 'Nd', 'Sc', 'Ga', 'Ru', 'Ce'} 33
[]
['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'P', 'S', 'Cl', 'K', 'Ca', 'Ti', 'V', 'Cr', 'Mn', 'Cu', 'Zn', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Tc', 'Ag', 'Cd', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'Pm', 'Hf', 'Ta', 'W', 'Re', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Pa', 'Np', 'Pu']


In [15]:
formula_unitcell['comp_fractions'] = formula_unitcell['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
formula_unitcell

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp,nelems,comp_dict,comp_fractions
0,CeGa,210.590,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,CeGa,215.020,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,CeGa,215.030,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1448,NiPt,26.473,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1449,SnYb,54.147,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1450,SnYb,54.800,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
print (formula_unitcell[:1])
#oqmd_data['solution_term'] = oqmd_data['comp_obj'].apply(lambda x: compute_mixing_term(x))
print (formula_unitcell.shape)
formula_unitcell = formula_unitcell[~formula_unitcell['comp_fractions'].isnull()]
print (formula_unitcell.shape)

  composition  Unit cell volume, A^3  comp_obj pretty_comp  nelems  \
0        CeGa                 210.59  (Ce, Ga)        CeGa       2   

                comp_dict                                     comp_fractions  
0  {'Ce': 0.5, 'Ga': 0.5}  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
(1452, 7)
(1452, 7)


In [17]:
for i,e in enumerate(elements_tl):
    formula_unitcell[e] = [ x[i] for x in formula_unitcell['comp_fractions']]
formula_unitcell.shape

(1452, 93)

In [18]:
formula_unitcell.columns

Index(['composition', 'Unit cell volume, A^3', 'comp_obj', 'pretty_comp',
       'nelems', 'comp_dict', 'comp_fractions', 'H', 'Li', 'Be', 'B', 'C', 'N',
       'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti',
       'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
       'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd',
       'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce',
       'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
       'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb',
       'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu'],
      dtype='object')

In [19]:
feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.IonProperty(fast=True)])

In [20]:
#feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                                         # cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

#Magpie folder is empty...

In [21]:
feature_labels = feature_calculators.feature_labels()

In [22]:
print (feature_labels, len(feature_labels)) #9/145 are not magpie-based

['0-norm', '2-norm', '3-norm', '5-norm', '7-norm', '10-norm', 'compound possible', 'max ionic char', 'avg ionic char'] 9


In [23]:
feature_calculators.featurize_dataframe(formula_unitcell, col_id='comp_obj');
formula_unitcell

MultipleFeaturizer:   0%|          | 0/1452 [00:00<?, ?it/s]

,composition,"Unit cell volume, A^3",comp_obj,pretty_comp,nelems,comp_dict,comp_fractions,H,Li,Be,...,Pu,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,compound possible,max ionic char,avg ionic char
0,CeGa,210.590,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.112214,0.028054
1,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.112214,0.028054
2,CeGa,212.790,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.112214,0.028054
3,CeGa,215.020,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.112214,0.028054
4,CeGa,215.030,"(Ce, Ga)",CeGa,2,"{'Ce': 0.5, 'Ga': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.112214,0.028054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,NiPt,26.500,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.033646,0.008411
1448,NiPt,26.473,"(Ni, Pt)",NiPt,2,"{'Ni': 0.5, 'Pt': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.033646,0.008411
1449,SnYb,54.147,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.168813,0.042203
1450,SnYb,54.800,"(Sn, Yb)",YbSn,2,"{'Yb': 0.5, 'Sn': 0.5}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,...,0.0,2,0.707107,0.629961,0.574349,0.552045,0.535887,False,0.168813,0.042203
